# 進階檢索策略 - Part 4
## ReRank

### FlagEmbeddingReranker
這是用於在本地執行開源重新排序（reranker）模型的 **LlamaIndex 整合類別**。

**運作方式：**  
它是對 BAAI（北京智源人工智慧研究院）模型套件的封裝。該類別採用 **Cross-Encoder（交叉編碼器）架構**，也就是同時處理查詢（query）與文件（document），以計算「真正的」相關性分數，而不是比較預先計算好的向量。  

**最適合對象：**  
希望在自有硬體（GPU）上執行高品質重新排序、且不想支付 API 呼叫費用的開發者。

---

**BAAI/bge-reranker-large**

這是目前全球最受歡迎的開源重新排序模型之一。

**主要優勢：**  
它在 MTEB（Massive Text Embedding Benchmark，大規模文字嵌入基準測試）中長期名列前茅。「Large」版本（約 5.6 億參數）在英文與中文任務上都具有極高的準確度。  

**取捨：**  
由於它是 Cross-Encoder 架構，其速度明顯慢於初始的向量搜尋。因此，通常只會將前 10–50 筆文件傳入該模型進行重新排序。

**目前狀態：**  
較新的 **BGE-Reranker-v2-m3**（近期發布）功能更加全面，支援多語言並可處理極長的上下文內容。


In [ ]:
pip install git+https://github.com/FlagOpen/FlagEmbedding.git

In [ ]:
import os

os.chdir("../")

from llama_index.llms.openai import OpenAI
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker

from initialization import credential_init
from src.ollama_connection import llama_index_ollama

ollama_llm = llama_index_ollama(model="gpt-oss:120b-cloud", temperature=0)

credential_init()

flag_embedding_reranker = FlagEmbeddingReranker(
    top_n=20,
    model="BAAI/bge-reranker-large",
    # 如果你有 NVIDIA GPU，強烈建議將此項設為 True。使用 FP16（半精度）可以顯著減少顯存佔用並提升推論速度，且對重排精度影響極小。
    use_fp16=False
)

<!-- 向量檢索 $top\_k=50 \rightarrow$ 粗選 50 份 -->

**RAG 架構中的兩階段檢索與 `top_n` 參數說明**

在 **Retrieval-Augmented Generation（RAG）** 架構中，檢索（Retrieval）通常設計為「兩個階段」，而 `top_n` 是控制最終輸出品質與成本的核心參數。

檢索漏斗：
    
向量檢索 $top\_k=50 \rightarrow$ 粗選 50 份

ReRanker $top\_k=5 \rightarrow$ 精選 5 份

LLM Context $\rightarrow$ 最終處理 5 份

---

**一、兩階段檢索流程**

**第一階段：初篩（Candidate Retrieval）**

- 從向量資料庫中快速找回大量候選文檔（例如 30～100 份）
- 常用方法包含：
  - 向量相似度（Cosine / Dot Product）
  - ANN（Approximate Nearest Neighbor）
- 特性說明：
  - 優點：速度快、可擴展性高
  - 缺點：精度較低，可能包含語意相似但實際不相關的文檔

---

**第二階段：重排（Re-ranking）**

- 使用 Reranker 模型對第一階段的候選文檔進行深度語意分析
- 重新計算「問題（Query）」與「每一份文檔」之間的相關性分數
- 特性說明：
  - 優點：語意理解能力強、排序精準
  - 缺點：計算成本較高，因此僅用於少量候選文檔

---

**二、`top_n` 的定義**

`top_n` 用來決定：

> 最終要交給 LLM 作為上下文（Context）的文檔數量

**範例說明**

- 第一階段取回：50 份文檔  
- 經 Reranker 重新排序後  
- 設定 `top_n = 10`  

結果為：  
只保留「相關性最高的前 10 份文檔」傳送給 LLM 作為參考上下文。

---

**三、為什麼需要設定 `top_n`？**

**1. 精準過濾**
- 移除第一階段中：
  - 表面上相似
  - 但實際語意不相關的雜訊文檔

**2. 優化成本**
- 減少傳送給 LLM 的文字量
- 有效降低：
  - Token 使用量
  - 推論成本
  - 系統延遲（Latency）

**3. 提升生成品質**
- 避免上下文過長造成：
  - 關鍵資訊被稀釋
  - LLM 出現「資訊遺忘（Lost in the Middle）」問題

---

**四、`top_n` 設定建議**

| 使用情境 | 建議 `top_n` |
|---------|-------------|
| 一般問答（FAQ、客服） | 3 ～ 5 |
| 技術問題、需引用文件 | 5 ～ 8 |
| 複雜推理、多來源整合 | 10 以上 |

注意事項：  
`top_n` 並非設定得越大越好，應在「資訊完整性」與「上下文效率」之間取得平衡。

---

**總結**

- 第一階段：快但不精，負責「找得到」
- 第二階段：慢但精，負責「找得準」
- `top_n`：控制最終輸入給 LLM 的「黃金上下文數量」

合理設定 `top_n`，是打造高品質、低成本 RAG 系統的關鍵之一。


使用第二周的數據

In [ ]:
from glob import glob

from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter

documents = SimpleDirectoryReader(input_files=[f for f in glob("week_2/data/*.txt")]).load_data()

sentence_splitter_node_parser = SentenceSplitter(
    chunk_size=1024,
    chunk_overlap=20,
)

nodes = sentence_splitter_node_parser.get_nodes_from_documents(documents)

建立VectorStoreIndex

In [ ]:
import faiss
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core.callbacks import LlamaDebugHandler, CallbackManager

d = 1024 # 必須與 embedding model 的輸出維度一致

faiss_index = faiss.IndexFlatL2(d)
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

callback_manager = CallbackManager([LlamaDebugHandler()])

index = VectorStoreIndex(
        nodes,
        embed_model=embed_model,
        storage_context=storage_context,
        callback_manager=callback_manager,
    )

In [ ]:
print(len(index.index_struct.nodes_dict))

In [ ]:
rerank_query_engine = index.as_query_engine(
    similarity_top_k=50,
    node_postprocessors=[
        flag_embedding_reranker
    ],
    llm=ollama_llm
)

In [ ]:
rerank_response = rerank_query_engine.query("英雄")

In [ ]:
retrieved_nodes = rerank_response.source_nodes

In [ ]:
len(retrieved_nodes)

In [ ]:
retrieved_nodes[:5]

### GPTReranker

RankGPT 是一種**列表式重排序（Listwise Reranking）**策略，利用大型語言模型來對文件進行排序。

- 運作原理： 它並非逐一為文件評分，而是將查詢（Query）與一整組文件列表同時輸入至大型語言模型（LLM），並詢問：「這些文件中哪些最相關？請按順序輸出編號。」
- 核心優勢： 它能發揮前沿 LLM 的完整推理能力，比起小型的編碼器模型（Encoder-only models），它更能理解語意細微的差別、複雜邏輯以及使用者的意圖。
- 權衡考量： 成本較高（API Token 消耗）且延遲較長（需等待 LLM 生成結果）。它通常被視為高標準檢索流程中的「最後磨光（Final Polish）」步驟。

In [ ]:
from llama_index.postprocessor.rankgpt_rerank import RankGPTRerank

gpt_reranker = RankGPTRerank(llm=ollama_llm, top_n=3)

In [ ]:
rerank_query_engine = index.as_query_engine(
    similarity_top_k=50,
    node_postprocessors=[
        gpt_reranker
    ]
)

In [ ]:
rerank_response = rerank_query_engine.query("英雄")
retrieved_nodes = rerank_response.source_nodes

In [ ]:
retrieved_nodes

除了開源的 BGE 方案，若追求極致的理解能力或企業級的穩定性，我們可以考慮基於 LLM 的 RankGPT 或託管型的 Cohere API。

### Cohere Rerank 4

Cohere Rerank 4 (正式名稱為 Rerank v4.0) 是 Cohere 於 2025 年底推出的最新一代重排序模型。與前代（v3.5）相比，它在效能、上下文長度和企業級應用上都有顯著的突破，被視為目前生產環境中最領先的託管型重排序解決方案之一。

以下是 Rerank 4 的關鍵特性：

1. 雙版本模型策略
Rerank 4 採取了與 LLM 類似的策略，提供了兩款針對不同需求優化的版本：

Rerank 4 Pro: 最強性能版本，專為追求極致準確度的複雜任務設計（如金融、法律、醫療）。在多項基準測試中其排名均位居前二。

Rerank 4 Fast: 效能平衡版本，在維持比 v3.5 更高準確度的同時，大幅降低了延遲（約比 Pro 快 30-40%），適合高流量與即時性需求。

2. 上下文窗口巨大飛躍 (32k Tokens)
從 4k 到 32k： 相比前代 v3.5 的 4096 tokens，Rerank 4 的上下文長度提升了 8 倍。

3. 多語言與跨語言能力
支持 100+ 種語言： 在中文、日文、德文、阿拉伯文等主流語言的表現達到了業界領先水平。

跨語言檢索： 它能極其精準地處理「英文查詢、中文文件」這類跨語言的相關性匹配。

4. 結構化數據與推理能力
原生支持 YAML/JSON： 它可以直接對結構化數據進行排序，而不僅僅是純文本。

更強的推理： 針對需要邏輯推理的查詢（例如：需要綜合多個條件的搜尋），Rerank 4 的理解能力顯著優於傳統的編碼器模型。

5. 易於整合
API 接口： 只需幾行程式碼即可接入（支援 LlamaIndex、LangChain 等框架）。

總結來說： 如果您正在開發一個企業級 RAG 系統，且對多語言支持、長文件處理或金融/醫療數據精準度有很高要求，Cohere Rerank 4 Pro 是目前的最佳選擇；如果您更在意成本與回覆速度，Rerank 4 Fast 則提供了極佳的性價比。

**可以在 https://cohere.com/ 註冊一個帳號，取得試用API KEY。

In [ ]:
from llama_index.postprocessor.cohere_rerank import CohereRerank

cohere_reranker = CohereRerank(api_key=os.environ["COHERE_API_KEY"], top_n=10)

rerank_query_engine = index.as_query_engine(
    similarity_top_k=50,
    node_postprocessors=[
        cohere_reranker
    ],
    llm=ollama_llm
)

In [ ]:
rerank_response = rerank_query_engine.query("英雄")
retrieved_nodes = rerank_response.source_nodes

#### ReRank 策略的比較表

| 方案 | 成本 | 速度 | 適用場景 | 
| --- | --- | --- | --- |
| BGE (Local) | 免費 (需 GPU) | 快 | 隱私要求高、預算有限 |
| RankGPT | 高 (API 消耗) | 慢 | 極致準確度、長文本理解 | 
| Cohere v4 | 中 (按次收費) | 極快 | 企業生產環境、多語言混合 |


剛才我們學習了如何對現有的結果進行『重新排序』，但如果一開始向量搜尋就沒找對內容（例如關鍵字精確匹配失敗），那重新排序也沒救。因此，接下來我們要進入 Hybrid Retriever（混合檢索），學習如何結合傳統搜尋與語意搜尋的優勢...

### 套用多個ReRank策略

為什麼要使用「複數 ReRank」

----------------------------

使用複數 ReRank，本質上是將檢索與排序拆成「由粗到細」的多階段流程，讓系統在效率、成本與準確度之間取得更好的平衡。

---

1. 先快篩，再精排（提升效率與降低成本）

第一層 ReRank 通常使用速度快、成本低的方法，先排除明顯不相關的節點；  
後續 ReRank 則使用較精準但較慢或較昂貴的模型，專注在少量高潛力內容上。

這樣可以避免一開始就讓昂貴模型處理大量低品質資料。

---

2. 同時兼顧 Recall 與 Precision

單一 ReRank 常面臨兩難：

- top_n 設太小，容易過早刪掉真正相關的 chunk  
- top_n 設太大，排序成本高且雜訊多  

多層 ReRank 的做法是：

- 前段保留較多結果，確保 Recall  
- 後段逐步縮小範圍，提高 Precision  

---

3. 結合不同排序訊號，結果更穩定

不同 ReRank 關注的訊號可能不同，例如：

- 向量語意相似度  
- 關鍵字或詞彙重疊  
- 問題與內容的回答適配度（LLM-based ReRank）  

透過多層 ReRank，可以從不同角度反覆確認內容相關性，降低單一方法誤判的風險。

---

4. 提供更乾淨的 Context 給 LLM

經過多階段篩選後：

- 節點數量更少  
- 不相關內容更少  
- Context 更集中  

這能幫助 LLM 生成更準確、較少 hallucination 的回答。

---

一句話總結

複數 ReRank 就像「初選 → 複選 → 決選」，  
不是一次就做出最終判斷，而是逐步收斂到最相關的結果。


In [ ]:
!pip install arize-phoenix

In [ ]:
import phoenix as px
import llama_index.core

# 啟動本地追蹤伺服器
px.launch_app()
llama_index.core.set_global_handler("arize_phoenix")

In [ ]:
ensemble_rerank_query_engine = index.as_query_engine(
    similarity_top_k=50,
    node_postprocessors=[
        flag_embedding_reranker,
        cohere_reranker,
        gpt_reranker
    ],
    llm=ollama_llm
)

In [ ]:
rerank_response = ensemble_rerank_query_engine.query("誰是最強的禿頭?")

## Hybrid Retriever

ReRank 解決的是『排不準』的問題，但如果向量檢索一開始就『找不到』（Recall 召回不足），則需要 Hybrid Retriever 來補足。

### BM25

為了讓大家專注於 BM25 的機制而非處理中文斷詞的繁瑣配置，接下來的 Hybrid 範例我們先將切換至英文資料集（Tech Companies）。

然後再來實驗如何加入中文Tokenization (參考https://zhuanlan.zhihu.com/p/1911447708331376894)

In [ ]:
from pathlib import Path

import wikipediaapi


wiki_titles = ["Anthropic", "OpenAI", "XAI_(company)", "Tesla,_Inc."]
wiki_metadatas = {
    "Anthropic": {
        "headquarter": "San Francisco",
        "industry": "Artificial intelligence"
    },
    "OpenAI": {
        "headquarter": "San Francisco",
        "industry": "Artificial intelligence"
    },
    "XAI_(company)": {
        "headquarter": "Palo Alto",
        "industry": "Technology"
    },
    "Tesla,_Inc.": {
        "headquarter": "Austin",
        "industry": "Automotive"
    },
}

wiki_wiki = wikipediaapi.Wikipedia(user_agent='AI Tutorial(mengchiehling@gmail.com)', language='en')

In [ ]:
for title in wiki_titles:
    page = wiki_wiki.page(title)
    wiki_text = page.text

    data_path = Path("week_4/data")
    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path / f"{title}.txt", "w", encoding="utf-8") as fp:
        fp.write(wiki_text)

In [ ]:
documents = SimpleDirectoryReader(input_files=[f for f in glob("week_4/data/*.txt")]).load_data()

In [ ]:
from llama_index.core import Document
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter

# initialize node parser
splitter = SentenceSplitter(chunk_size=512)

nodes = splitter.get_nodes_from_documents(documents)

In [ ]:
len(nodes)

### 使用 BM25Retriever 的兩種方式

BM25Retriever 可以透過不同來源建立，以下介紹兩種常見做法。

**方法一：直接從 nodes 建立 BM25Retriever（可存取到磁碟）**

這種方式是直接使用 nodes 來建立 retriever，適合資料量不大、或不需要遠端儲存的情境。

In [ ]:
import Stemmer
from llama_index.retrievers.bm25 import BM25Retriever

# 直接從 nodes 建立 BM25Retriever
bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=5,
    # 可選：設定 stemmer 與 stopwords 語言
    # 用於移除停用詞並對查詢與文本做 stemming
    # 預設語言為 english
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)


**方法二：使用 docstore 管理 nodes（支援遠端儲存）**

這種方式會先將 nodes 存入 docstore，再由 docstore 建立 BM25Retriever。

優點是 docstore 可以是遠端的，例如 MongoDB、Redis 等，適合大型或分散式系統。若要真正實現分佈式，需對接 RedisDocumentStore 或 MongoDocumentStore

In [ ]:
import Stemmer
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.retrievers.bm25 import BM25Retriever

# 建立 docstore 並加入 nodes
docstore = SimpleDocumentStore()
docstore.add_documents(nodes)

# 從 docstore 建立 BM25Retriever
bm25_retriever = BM25Retriever.from_defaults(
    docstore=docstore,
    similarity_top_k=5,
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)


In [ ]:
# will retrieve context from specific companies
retrieved_nodes = bm25_retriever.retrieve(
    "artiticial intelligence"
)

In [ ]:
len(retrieved_nodes)

In [ ]:
retrieved_nodes[0]

**用 StorageContext persist docstore**

In [ ]:
# 建立 StorageContext
storage_context = StorageContext.from_defaults(
    docstore=docstore
)

# Persist 到磁碟
storage_context.persist(persist_dir="week_4/storage")


**重新載入（load）並建立 BM25Retriever**

In [ ]:
storage_context = StorageContext.from_defaults(
    persist_dir="week_4/storage"
)

docstore = storage_context.docstore

# 使用已載入的 docstore 建立 BM25Retriever
bm25_retriever_loaded = BM25Retriever.from_defaults(
    docstore=docstore,
    similarity_top_k=5,
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)

retrieved_nodes = bm25_retriever.retrieve(
    "artiticial intelligence"
)

In [ ]:
len(retrieved_nodes)

### BM25 Retriever + MetadataFiltering

In [ ]:
# Define metadata filters
from llama_index.core.vector_stores.types import (
    MetadataFilters,
    MetadataFilter,
    FilterOperator,
    FilterCondition,
)

filters = MetadataFilters(
    filters=[
        MetadataFilter(
            key="headquarter",
            value="San Francisco",
            operator=FilterOperator.EQ,
        )
    ],
    condition=FilterCondition.AND,
)

使用一個dummy query 來測試filters是否成功

In [ ]:
retrieved_nodes = BM25Retriever.from_defaults(
    docstore=docstore,
    similarity_top_k=3,
    filters=filters,  # Add filters here
    stemmer=Stemmer.Stemmer("english"),
    language="english",
).retrieve("Hello, world!")

可以拿來嘗試的query:

**Company Background & Positioning**

> Best for testing keyword matching + semantic understanding

- What is OpenAI and what is its core mission?
- How does Anthropic position itself in the AI safety landscape?
- What company is xAI and why was it founded?
- What is Tesla primarily known for besides electric vehicles?

---

**Founders & Key Figures**

> Strong keyword signals (names, dates) with semantic reinforcement

- Who founded OpenAI and when?
- Who are the founders of Anthropic?
- What is Elon Musk’s involvement in xAI?
- How is Elon Musk connected to Tesla?

---

**Mission, Goals, and Philosophy**

> Conceptual questions where dense retrieval excels

- What does OpenAI mean by artificial general intelligence?
- What is Anthropic’s approach to building safe AI systems?
- What problem is xAI trying to solve?
- What is Tesla’s long-term vision for sustainable energy?

---

**Products & Technologies**

> Hybrid retrieval works especially well for mixed factual + semantic queries

- What products or models has OpenAI released?
- What is Anthropic’s Claude model?
- Does xAI develop large language models?
- What technologies does Tesla develop besides cars?

---

**Comparison Queries**

> Ideal for demonstrating QueryFusionRetriever and multi-document reasoning

- How is OpenAI different from Anthropic?
- Compare the goals of OpenAI and xAI.
- How do AI companies like OpenAI and Anthropic differ from Tesla?
- Which of these companies focuses on AI safety?

---

**Advanced / RAG Demo Queries (Recommended)**

> Excellent for end-to-end RetrieverQueryEngine demonstrations

- Which companies in the documents are focused on artificial intelligence research?
- Which organization emphasizes AI safety and alignment?
- What companies were founded by Elon Musk?
- Summarize the missions of OpenAI, Anthropic, xAI, and Tesla.

### 中文 Tokenization

Byte Pair Encoding (BPE)

- pip install jieba

參考Blog https://medium.com/seaniap/python-%E4%B8%AD%E6%96%87%E8%87%AA%E7%84%B6%E8%AA%9E%E8%A8%80%E8%99%95%E7%90%86jieba%E6%96%B7%E8%A9%9E-9ab44a1b6a9d 

In [ ]:
import jieba

# https://github.com/fxsjy/jieba/blob/master/extra_dict/dict.txt.big
"""
jieba 預設不支援繁體字典，建議加入 jieba.set_dictionary('dict.txt.big') 的提示，這對台灣使用者更友好。
"""
jieba.set_dictionary('week_4/dict.txt.big')

# 在簡中的表現上比繁中要好，但目前也沒更好的選擇

# text = "清晨的阳光透过窗帘，在地板上洒下斑驳的光影。"
text = "清晨的陽光透過窗簾，在地板上灑下斑駁的光影。"

jieba.lcut_for_search(text)

In [ ]:
import jieba
import bm25s
from bm25s.tokenization import Tokenized
from typing import List, Union, Callable
from nltk import corpus
from tqdm.auto import tqdm

def tokenize(
    texts: Union[str, List[str]],
    lower: bool = True,
    token_pattern: str = r"(?u)\b\w\w+\b",
    stopwords: Union[str, List[str]] = "zh",
    stemmer: Callable = None,  
    return_ids: bool = True,
    show_progress: bool = True,
    leave: bool = False,
    allow_empty: bool = True,
) -> Union[List[List[str]], Tokenized]:
    if isinstance(texts, str):
        texts = [texts]

    language_map = {
        "zh":"chinese",
        "en":"english"
    }
    
    if isinstance(stopwords, str):
        stopword_list = corpus.stopwords.words(language_map[stopwords])
    else:
        stopword_list = []
        for stopword in stopwords:
            stopword_list.extend(corpus.stopwords.words(language_map[stopword]))
    corpus_ids = []
    token_to_index = {}
    for text in tqdm(
        texts, desc="Split strings", leave=leave, disable=not show_progress
    ):  
        if lower:
            text = text.lower()

        splitted = jieba.lcut_for_search(text)
        if allow_empty is False and len(splitted) == 0:
            splitted = [""]

        doc_ids = []
        for token in splitted:
            if token in stopword_list:
                continue 
            token_to_index.setdefault(token, len(token_to_index))
            doc_ids.append(token_to_index[token])
        corpus_ids.append(doc_ids)
    if return_ids:
        return Tokenized(ids=corpus_ids, vocab=token_to_index)
    else:
        unique_tokens = list(token_to_index.keys())
        for i, token_ids in enumerate(
            tqdm(
                corpus_ids,
                desc="Reconstructing token strings",
                leave=leave,
                disable=not show_progress,
            )
        ):
            corpus_ids[i] = [unique_tokens[token_id] for token_id in token_ids]

        return corpus_ids

In [ ]:
bm25s.tokenize = tokenize

Judgement Day!

import nltk

nltk.download('stopwords')

In [ ]:
import nltk

nltk.download('stopwords')

from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.retrievers.bm25 import BM25Retriever

documents = [
    Document(text="清晨的陽光透過窗簾，在地板上灑下斑駁的光影。"),
    Document(text="街角的咖啡店飄來陣陣香氣，引得路人頻頻駐足。"),
    Document(text="她翻開相冊，指尖輕輕摩挲著泛黃的照片片段。"),
    Document(text="一只貓懶洋洋地趴在窗台上，眯著眼打量過往的行人。")
]

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=2,
    # tokenizer=chinese_tokenize,
    language='zh'
)

nodes = retriever.retrieve("陽光")
for node in nodes:
    print(node)

## 混合式檢索器 (QueryFusionRetriever)

混合式檢索器結合了 BM25 與向量檢索（如 FAISS）的優點，能有效彌補彼此的限制：

- **提升召回率（Recall）**
  - BM25 擅長精確的關鍵字比對
  - FAISS 擅長捕捉語意相近但用詞不同的內容
  - 兩者結合能找回更多「真正相關」的文件

- **兼顧語意與關鍵字精準度**
  - 對於專有名詞、數字、程式碼等，BM25 表現較佳
  - 對於語意相似、同義改寫的查詢，向量檢索更有優勢

- **降低單一檢索方式的盲點**
  - 僅使用 BM25 可能忽略語意相關但未出現關鍵字的內容
  - 僅使用向量檢索可能在精確字詞匹配上表現不足

- **透過 Query Fusion 提升排序品質**
  - QueryFusionRetriever 可以將多種檢索結果加權、融合與重新排序
  - 提供更穩定且高品質的最終結果

---

**Hybrid Retriever（BM25 + Semantic Search）** 是一種同時結合  
👉 **關鍵字精準匹配** 與  
👉 **語意理解能力** 的檢索策略。

透過 **QueryFusionRetriever** 將結果整合後，再搭配 **RetrieverQueryEngine**，  
可以打造出更準確、更健壯、也更適合實務應用的 RAG（Retrieval-Augmented Generation）系統。

In [ ]:
import os

os.chdir("../")

建立語意檢索器

In [ ]:
from glob import glob

import faiss
from llama_index.core import StorageContext, SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.faiss import FaissVectorStore

d = 1024 # 必須與 embedding model 的輸出維度一致

faiss_index = faiss.IndexFlatL2(d)
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

documents = SimpleDirectoryReader(input_files=[f for f in glob("week_2/data/*.txt")]).load_data()

# initialize node parser
splitter = SentenceSplitter(chunk_size=512)

nodes = splitter.get_nodes_from_documents(documents)

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")


In [ ]:
from llama_index.core.callbacks import LlamaDebugHandler, CallbackManager

callback_manager = CallbackManager([LlamaDebugHandler()])

In [ ]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex(nodes=nodes, 
                         storage_context=storage_context,
                         callback_manager=callback_manager,
                         embed_model=embed_model)

In [ ]:
import bm25s
import jieba

from src.ollama_connection import llama_index_ollama
from src.tokenizer import chinese_tokenize

jieba.set_dictionary('week_4/dict.txt.big')
bm25s.tokenize = chinese_tokenize

ollama_llm = llama_index_ollama(model="gpt-oss:120b-cloud", temperature=0)

快速測試 BM25 是否正常

In [ ]:
from llama_index.core import Document
from llama_index.retrievers.bm25 import BM25Retriever

documents = [
    Document(text="清晨的陽光透過窗簾，在地板上灑下斑駁的光影。"),
    Document(text="街角的咖啡店飄來陣陣香氣，引得路人頻頻駐足。"),
    Document(text="她翻開相冊，指尖輕輕摩挲著泛黃的照片片段。"),
    Document(text="一只貓懶洋洋地趴在窗台上，眯著眼打量過往的行人。")
]

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=2,
    # tokenizer=chinese_tokenize,
    language='zh'
)

nodes = retriever.retrieve("陽光")
for node in nodes:
    print(node)

使用 QueryFusionRetriever 整合 兩個不同的檢索器

In [ ]:
import nest_asyncio

nest_asyncio.apply()

from llama_index.core.retrievers import QueryFusionRetriever

retriever = QueryFusionRetriever(
    [
        index.as_retriever(similarity_top_k=2),
        BM25Retriever.from_defaults(
            docstore=index.docstore, similarity_top_k=2,
            language='zh'
        ),
    ],
    num_queries=1,
    use_async=True,
    llm=ollama_llm
)

In [ ]:
nodes = retriever.retrieve("草帽海賊團的人員有哪些?")

In [ ]:
nodes

**num_queries**：

在這個步驟中，我們會將多個索引融合成一個單一的檢索器（retriever）。這個檢索器同時也會透過產生與原始問題相關的額外查詢來擴充（augment）你的查詢，並彙總所有查詢的結果。

此設定會執行 4 次查詢：一次使用你的原始查詢，並另外產生 3 個新的查詢。

預設情況下，它會使用以下提示（prompt）來產生額外的查詢：

In [ ]:
"""
QUERY_GEN_PROMPT = (
    "You are a helpful assistant that generates multiple search queries based on a "
    "single input query. Generate {num_queries} search queries, one on each line, "
    "related to the following input query:\n"
    "Query: {query}\n"
    "Queries:\n"
)
"""

In [ ]:
retriever = QueryFusionRetriever(
    [
        index.as_retriever(similarity_top_k=2),
        BM25Retriever.from_defaults(
            docstore=index.docstore, similarity_top_k=2,
            language='zh'
        ),
    ],
    num_queries=4,
    use_async=True,
    llm=ollama_llm
)

#### 深度解析：從多個維度捕捉語意（RAG-Fusion）

這裡設置的 num_queries 實際上體現了 RAG-Fusion 的核心思想。

克服查詢的不確定性：原始查詢往往較為簡短或模糊，而 RAG-Fusion 利用 LLM 將單一查詢拆解、改寫成多個不同角度的搜尋指令。

多角度檢索：例如，當使用者問「英雄」時，LLM 可能會產生「英雄定義」、「動漫中的英雄角色」或「英雄相關劇情」等查詢，確保從向量空間中檢索出更全面的上下文。

消除排序偏差：透過產生多個查詢並配合 Reciprocal Rank Fusion (RRF) 演算法，系統能有效減少單一檢索演算法對特定關鍵字的排序偏見，使最終結果更具魯棒性（Robustness）。

In [ ]:
nodes = retriever.retrieve("草帽海賊團的人員有哪些?")

In [ ]:
len(nodes)

將檢索器放入QueryEngine裡

In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever)

Fusion Retriever 中有一個參數:

mode: llama_index.core.retrievers.fusion_retriever.FUSION_MODES

來給不同檢索器給出的排名中的nodes重新進行混合排名

1. Reciprocal Rerank（RRF）

**模式（Mode）**  
FUSION_MODE.RECIPROCAL_RANK（預設）

**運作方式（How it works）**  
這是目前最常見、也最受歡迎的方法。  
它不會使用 LLM 來對文件進行評分，而是透過一個數學公式，根據文件在多個結果清單中的排名（rank）來重新排序。

**公式（Formula）**  

$score = sum( 1 / (k + rank(d)) )$

其中 k 是一個常數（通常為 60）。

**適用情境（Best for）**  
通用型的混合搜尋（Hybrid Search）。  
當你希望在向量搜尋與關鍵字搜尋之間取得良好平衡，同時避免額外的 LLM 成本時，非常適合使用。

---

2. Relative Score Fusion（相對分數融合）

**模式（Mode）**  
FUSION_MODE.RELATIVE_SCORE

**運作方式（How it works）**  
此方法不再關注文件的排名（例如第 1 名、第 2 名），而是直接使用實際的相似度分數。  
它會將來自不同檢索器（通常分數尺度不同）的分數正規化到 0 到 1 的範圍內，接著再對這些分數取平均。

**適用情境（Best for）**  
當檢索器所提供的信心分數（confidence score）本身就相當可靠時。

---

3. Distribution-Based Score Fusion（分佈式分數融合）

**模式（Mode）**  
FUSION_MODE.DIST_BASED_SCORE

**運作方式（How it works）**  
這是一種更進階的分數融合方法，會同時考慮結果集中分數的平均值（mean）與標準差（standard deviation）。  
這有助於處理不同檢索器可能產生「分數非常集中」或「分數分佈非常分散」的情況。

---

4. Simple Fusion（簡單融合）

**模式（Mode）**  
FUSION_MODE.SIMPLE

**運作方式（How it works）**  
這是一種基礎做法，通常只是將結果直接串接（concatenation）或進行簡單加總。  
相較於 RRF，這種方法在實際生產環境中較少使用，但可用於測試基準效能（baseline performance）。
